In [1]:
import pandas as pd
import numpy as np
import sys

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

import time

from pathlib import Path
import os

import datetime
import pickle 

## set up for imports of .py modules by adding path to sys.path

In [2]:
path = Path(os.getcwd())
path = str(path)
print(path)
sys.path.insert(1, path)

C:\Users\disle\Documents\Supervised_ML\final_project


In [3]:
import utils.numerical_attr_eda_utils as num_eda_utils
import utils.categorical_attr_eda_utils as cat_eda_utils
import utils.all_attr_eda_utils as all_attr_eda_utils
import utils.attr_eda_utils as attr_eda_utils
import utils.assign_and_lab_utils as al_utils
import utils.multi_class_target_encoder_utils as mc_te_utils
import utils.classification_utils as class_utils
import utils.classifier_hyp_param_grid as cl_hpg

## set up to time script run time

In [4]:
start = time.time()

## parameters

In [5]:
path_to_data = 'data/genetic_disorder.csv'

train_test_split_random_state = 42
train_validation_split_random_state = 42
fast_script_dev = False  
model_random_state = 42
test_size = 0.20
target_attr = 'Genetic Disorder'
prediction_task_type = 'classification'
sgd_max_iter = 10000
binary = False
missingness_threshold = 0.20
calibrate_classifiers = True

## import data

In [13]:
genetic_df = pd.read_csv(path_to_data)
print(genetic_df.shape)
genetic_df.head()

(22083, 45)


,Patient Id,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Patient First Name,Family Name,Father's name,...,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
0,PID0x6418,2.0,Yes,No,Yes,No,4.760603,Richard,NaN,Larre,...,NaN,9.857562,NaN,1.0,1.0,1.0,1.0,1.0,Mitochondrial genetic inheritance disorders,Leber's hereditary optic neuropathy
1,PID0x25d5,4.0,Yes,Yes,No,No,4.910669,Mike,NaN,Brycen,...,Multiple,5.522560,normal,1.0,NaN,1.0,1.0,0.0,NaN,Cystic fibrosis
2,PID0x4a82,6.0,Yes,No,No,No,4.893297,Kimberly,NaN,Nashon,...,Singular,NaN,normal,0.0,1.0,1.0,1.0,1.0,Multifactorial genetic inheritance disorders,Diabetes
3,PID0x4ac8,12.0,Yes,No,Yes,No,4.705280,Jeffery,Hoelscher,Aayaan,...,Singular,7.919321,inconclusive,0.0,0.0,1.0,0.0,0.0,Mitochondrial genetic inheritance disorders,Leigh syndrome
4,PID0x1bf7,11.0,Yes,No,NaN,Yes,4.720703,Johanna,Stutzman,Suave,...,Multiple,4.098210,NaN,0.0,0.0,0.0,0.0,NaN,Multifactorial genetic inheritance disorders,Cancer


In [14]:
#Drop patient Id column
genetic_df.drop('Patient Id', axis=1, inplace=True)

## drop observations with nans in the target attribute

### this can be out of pipeline because when the trained composite estimator predicts on unseen data there is no target attribute

In [15]:
print(genetic_df.shape)
genetic_df = genetic_df.dropna(subset=target_attr)
print(genetic_df.shape)

(22083, 44)
(19937, 44)


## convert target_attr to numerical encoding using label encoder

In [16]:
le = LabelEncoder()
# Encode 'Disorder_subclass'
#genetic_df['Disorder_subclass'] = le.fit_transform(genetic_df['Disorder_subclass'])

# Encode 'Genetic_disorder'
genetic_df['Genetic Disorder'] = le.fit_transform(genetic_df['Genetic Disorder'])
genetic_df.head()

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Patient First Name,Family Name,Father's name,Mother's age,...,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
0,2.0,Yes,No,Yes,No,4.760603,Richard,NaN,Larre,NaN,...,NaN,9.857562,NaN,1.0,1.0,1.0,1.0,1.0,0,Leber's hereditary optic neuropathy
2,6.0,Yes,No,No,No,4.893297,Kimberly,NaN,Nashon,41.0,...,Singular,NaN,normal,0.0,1.0,1.0,1.0,1.0,1,Diabetes
3,12.0,Yes,No,Yes,No,4.705280,Jeffery,Hoelscher,Aayaan,21.0,...,Singular,7.919321,inconclusive,0.0,0.0,1.0,0.0,0.0,0,Leigh syndrome
4,11.0,Yes,No,NaN,Yes,4.720703,Johanna,Stutzman,Suave,32.0,...,Multiple,4.098210,NaN,0.0,0.0,0.0,0.0,NaN,1,Cancer
5,14.0,Yes,No,Yes,No,5.103188,Richard,NaN,Coleston,NaN,...,Multiple,10.272230,normal,1.0,0.0,0.0,1.0,0.0,2,Cystic fibrosis


## train/test split

In [17]:
train_cap_x_df, train_y_df = \
    al_utils.perform_the_train_test_split(
    genetic_df, 
    test_size, 
    train_test_split_random_state, 
    val=False,
    stratify=True
)

ValueError: Input contains NaN

In [ ]:
del df

## train/validation split

In [ ]:
train_cap_x_df, train_y_df = \
    al_utils.perform_the_train_test_split(
        pd.concat([train_cap_x_df, train_y_df], axis=1), 
        test_size, 
        train_validation_split_random_state, 
        val=True,
        stratify=True
)

## drop attributes with missingness above threshold

In [ ]:
missingness_drop_list = []
for attr in train_cap_x_df.columns:
    attr_missingness = train_cap_x_df[attr].isna().sum() / train_cap_x_df.shape[0]
    if attr_missingness >= missingness_threshold:
        missingness_drop_list.append(attr)

missingness_drop_list

## identify non machine learning attributes

### these are attributes that are not meaningful to machine learning - examples include observation identification attributes, etc.

In [ ]:
train_cap_x_df.head()

In [ ]:
concern_list = all_attr_eda_utils.check_for_complete_unique_attrs(train_cap_x_df)
print(f'\nconcern_list:\n{concern_list}', sep='')

In [ ]:
non_ml_attr_list = ['attr_0']

## identify attributes to drop from machine learning 

### these are attributes that were candidates for machine learning but you have chosen to eliminate from machine learning - elimination can be due to vifs, etc.

In [ ]:
train_cap_x_df.columns

In [ ]:
ml_attr_drop_list = []

## establish machine learning attribute configuration

#### get all numerical attributes

In [ ]:
train_cap_x_df.select_dtypes(include=np.number).columns

#### get all object attributes - these are presumed to be nominal attributes

In [ ]:
train_cap_x_df.select_dtypes(include=object).columns

#### assign the attributes

In [ ]:
print(f'train_cap_x_df.shape[1]: {train_cap_x_df.shape[1]}')

ml_ignore_list = missingness_drop_list + non_ml_attr_list + ml_attr_drop_list
print('\n', ml_ignore_list)

nominal_attr = ['attr_3', 'attr_6', 'attr_10', 'attr_12', 'attr_14']
print('\n', nominal_attr)

numerical_attr = ['attr_1', 'attr_2', 'attr_4', 'attr_7', 'attr_8', 'attr_9', 'attr_11', 'attr_13', 'attr_15']
print('\n', numerical_attr)

assert(train_cap_x_df.shape[1] == len(ml_ignore_list) + len(nominal_attr) + len(numerical_attr))  # got them all?

print('\n', len(numerical_attr) + len(nominal_attr))
print('\n', numerical_attr + nominal_attr)

## assess target attribute unbalanced

In [ ]:
train_y_df[target_attr].unique().tolist()

In [ ]:
train_y_df[target_attr].value_counts(normalize=True)

## steps to deal with unbalanced classes

To be completed

## survey/evaluate default composite estimators with ranking metrics

### define the estimators involved in the experiment

####  use default instantiations (except for random_state, class_weight and a few others as noted below)

In [ ]:
estimator_names = [
    'SGDClassifier', 
    'DecisionTreeClassifier', 
    #'RandomForestClassifier', 
    #'AdaBoostClassifier', 
    #'GradientBoostingClassifier'
]

estimator_list = [
    
    SGDClassifier(loss='log_loss', random_state=model_random_state, class_weight='balanced',
                  max_iter=sgd_max_iter),  # logistic regr
    
    DecisionTreeClassifier(criterion='log_loss', random_state=model_random_state, class_weight='balanced'),
    
    #RandomForestClassifier(criterion='log_loss', random_state=model_random_state, 
                       #    class_weight='balanced_subsample'),
    
#     AdaBoostClassifier(
#         estimator=DecisionTreeClassifier(
#             criterion='log_loss', 
#             random_state=model_random_state, 
#             class_weight='balanced',
#             max_depth=1
#         ),
#         random_state=model_random_state
#     ),
    
#     GradientBoostingClassifier(loss='log_loss', random_state=model_random_state),
    
]

### fit the default default models and evaluate performance on the train set

In [ ]:
####################################################################
# class_eval_dict:
#    key = name of function in classification_utils.py
#    value = [bool, function kwargs]  bool = True then call function
print_plots = False
class_eval_dict={
    'binary': binary,
    'scoring': 'average_precision',
    'get_precision_recall_curves': [True, 
                                    {'print_prc': print_plots, 
                                     'print_prd': print_plots,
                                    }],
    'get_roc_curve': [True, 
                      {
                        'print_roc': print_plots,
                      }]
}

default_train_compare_df, trained_default_estimator_dict = \
    al_utils.fit_collection_of_estimators(
        numerical_attr, 
        nominal_attr, 
        estimator_names, 
        estimator_list, 
        train_cap_x_df, 
        train_y_df, 
        data_set_type='train', 
        model_selection_stage='default',
        prediction_task_type='classification',
        class_eval_dict=class_eval_dict
)
default_train_compare_df

### evaluate the performance of the trained default estimators on the validation set

In [ ]:
validation_df = pd.read_csv('validation_df.csv').set_index(keys='index')
validation_df.index.name = None
validation_cap_x_df, validation_y_df = validation_df.iloc[:, :-1], validation_df.iloc[:, -1].to_frame()

In [ ]:
# class_eval_dict:
#    key = name of function in classification_utils.py
#    value = [bool, function kwargs]  bool = True then call function
print_plots = False
class_eval_dict={
    'binary': binary,
    'scoring': 'average_precision',
    'get_precision_recall_curves': [True, 
                                    {'print_prc': print_plots, 
                                     'print_prd': print_plots,
                                    }],
    'get_roc_curve': [True, 
                      {
                        'print_roc': print_plots,
                      }]
}


default_validation_compare_df = al_utils.eval_trained_estimators_in_trained_estimator_dict_class(
    trained_default_estimator_dict, 
    validation_cap_x_df, 
    validation_y_df, 
    data_set_type='validation',
    model_selection_stage='default', 
    class_eval_dict=class_eval_dict
)
default_validation_compare_df

In [ ]:
del validation_cap_x_df, validation_y_df

### assemble a data frame of default estimator performance on the train and validation stage

In [ ]:
compare_df = pd.concat([default_train_compare_df, default_validation_compare_df], axis=0).\
    sort_values(['estimator', 'data_set_type', 'model_selection_stage'])
compare_df

### check out vifs of the design matrices for the default models

In [ ]:
al_utils.check_out_vifs_of_preprocessed_design_matrices_of_a_collection_of_trained_estimators(
    trained_default_estimator_dict, 
    train_cap_x_df, 
    data_set_type='train',
    model_selection_stage='default_instantiation'
)

## short list composite estimators based on default estimator findings

To be completed

## hyperparameters tuning on default models using GridSearchCV with ranking metrics

### design the hyperparameter tuning experiment to select the best model by populating the parameter grid with hyperparameter values

In [ ]:
alpha_points = 5
l1_ratio_points = 5
m_points = 5

hyp_param_tuning_exp_dict = cl_hpg.get_hyp_param_tuning_exp_dict(
    estimator_names,
    estimator_list, 
    alpha_points, 
    l1_ratio_points, 
    m_points, 
    train_cap_x_df, 
    binary=True,
    fast_script_dev=fast_script_dev, 
    print_param_grids=True
)

### perform a grid search over hyper parameters to select best model

In [ ]:
# class_eval_dict:
#    key = name of function in classification_utils.py
#    value = [bool, function kwargs]  bool = True then call function
print_plots = False
class_eval_dict={
    'binary': binary,
    'scoring': 'average_precision',
    'get_precision_recall_curves': [True, 
                                    {'print_prc': print_plots, 
                                     'print_prd': print_plots,
                                     'data_set_name': '',  # this is here to make things work - a bit of a hack
                                     'model_selection_stage': '',  # this is here to make things work - a bit of 
                                                                   # a hack
                                    }],
    'get_roc_curve': [True, 
                      {
                        'print_roc': print_plots,
                        'data_set_name': '',  # this is here to make things work - a bit of a hack
                        'model_selection_stage': '',  # this is here to make things work - a bit of a hack
                      }]
}

grid_search_cv_results_df, _ = \
    al_utils.grid_search_cv_wrapper(
        estimator_names,
        hyp_param_tuning_exp_dict, 
        numerical_attr, 
        nominal_attr,
        train_cap_x_df, 
        train_y_df, 
        target_attr,
        prediction_task_type='classification',
        class_eval_dict=class_eval_dict
)
grid_search_cv_results_df

## evaluate tuned composite estimators with ranking metrics - bootstrapping (no refit)

In [ ]:
validation_df = pd.read_csv('validation_df.csv').set_index(keys='index')
validation_df.index.name = None
validation_cap_x_df, validation_y_df = validation_df.iloc[:, :-1], validation_df.iloc[:, -1].to_frame()

In [ ]:
# class_eval_dict:
#    key = name of function in classification_utils.py
#    value = [bool, function kwargs]  bool = True then call function
print_plots = False
class_eval_dict={
    'binary': binary,
    'scoring': 'average_precision',
    'get_precision_recall_curves': [True, 
                                    {'print_prc': print_plots, 
                                     'print_prd': print_plots,
                                    }],
    'get_roc_curve': [True, 
                      {
                        'print_roc': print_plots,
                      }]
}

_ = \
    al_utils.execute_and_plot_bootstrap_eval_without_refit(
        estimator_names,
        grid_search_cv_results_df,
        validation_cap_x_df, 
        validation_y_df,
        num_bs_samples=20,
        model_selection_stage='tuned_instantiation',
        data_set_type='validation_set',
        prediction_task_type='classification',
        class_eval_dict=class_eval_dict
)

### visualize the model performance using ranking metrics

In [ ]:
class_perf_dict = \
    class_utils.ranking_metrics_class_perf_assess_binary(
        estimator_names, 
        grid_search_cv_results_df, 
        validation_cap_x_df, 
        validation_y_df, 
        classification_threshold=0.50, 
        cvs_compute=False, 
        cvs_print=False,
        data_set_name='validation', 
        model_selection_stage='tuned'
)

In [ ]:
del validation_cap_x_df, validation_y_df

## calibrate the composite estimators

In [ ]:
if calibrate_classifiers:
    validation_df = pd.read_csv('validation_df.csv').set_index(keys='index')
    validation_df.index.name = None
    validation_cap_x_df, validation_y_df = validation_df.iloc[:, :-1], validation_df.iloc[:, -1].to_frame()

#### split off some of the validation set for calibration

In [ ]:
if calibrate_classifiers:
    cal_split_size = 0.50
    cal_split_random_state = 21

    cal_cap_x_df, cal_y_df, validation_cap_x_df, validation_y_df = al_utils.split_validation_for_calibration(
        pd.concat([validation_cap_x_df, validation_y_df], axis=1),
        cal_split_size=cal_split_size,
        cal_split_random_state=cal_split_random_state
    )

#### perform the calibration

In [ ]:
if calibrate_classifiers:
    # class_eval_dict:
    #    key = name of function in classification_utils.py
    #    value = [bool, function kwargs]  bool = True then call function
    print_plots = False
    class_eval_dict={
        'binary': binary,
        'scoring': 'average_precision',
        'get_precision_recall_curves': [True, 
                                        {'print_prc': print_plots, 
                                         'print_prd': print_plots,
                                        }],
        'get_roc_curve': [True, 
                          {
                            'print_roc': print_plots,
                          }]
    }

    sig_cal_grid_search_cv_results_df = al_utils.calibrate_estimators(
        estimator_names, 
        grid_search_cv_results_df, 
        pd.concat([train_cap_x_df, cal_cap_x_df], axis=0),
        pd.concat([train_y_df, cal_y_df], axis=0),
        validation_cap_x_df, 
        validation_y_df,
        class_eval_dict=class_eval_dict,
        calibration_data_set_name='probability calibration', 
        validation_data_set_name='validation',
        model_selection_stage='tuned', 
        method='isotonic',  # 'sigmoid' or 'isotonic'
        ensemble=True
    )

    estimator_names = al_utils.get_estimator_names_helper(grid_search_cv_results_df, 
                                                          sig_cal_grid_search_cv_results_df)
    grid_search_cv_results_df = pd.concat([sig_cal_grid_search_cv_results_df, grid_search_cv_results_df], axis=0)

In [ ]:
if calibrate_classifiers:
    del cal_cap_x_df, cal_y_df 

In [ ]:
if calibrate_classifiers:
    del validation_cap_x_df, validation_y_df

## check for false discoveries

In [ ]:
validation_df = pd.read_csv('validation_df.csv').set_index(keys='index')
validation_df.index.name = None
validation_cap_x_df, validation_y_df = validation_df.iloc[:, :-1], validation_df.iloc[:, -1].to_frame()

In [ ]:
# class_eval_dict:
#    key = name of function in classification_utils.py
#    value = [bool, function kwargs]  bool = True then call function
print_plots = False
class_eval_dict={
    'binary': binary,
    'scoring': 'average_precision',
    'get_precision_recall_curves': [True, 
                                    {'print_prc': print_plots, 
                                     'print_prd': print_plots,
                                    }],
    'get_roc_curve': [True, 
                      {
                        'print_roc': print_plots,
                      }]
}

al_utils.avoiding_false_discoveries_class(
    estimator_names, 
    grid_search_cv_results_df, 
    train_cap_x_df, 
    train_y_df, 
    validation_cap_x_df, 
    validation_y_df, 
    num_samples=20, 
    class_eval_dict=class_eval_dict,
    data_set_name='validation', 
    model_selection_stage='tuned'
)

In [ ]:
del validation_cap_x_df, validation_y_df

## time to execute to this point in notebook

In [ ]:
end = time.time()
print(f'script run time: {(end - start)/60} minutes')

## stop notebook execution and select a model to promote if there is more then one model

In [ ]:
if len(estimator_names) > 1:
    sys.exit()

## select a model

In [ ]:
estimator_names

In [ ]:
best_model = 'cal_iso_T_DecisionTreeClassifier'

In [ ]:
estimator_names = [best_model]

## permutation feature importance

In [ ]:
perm_imp_dict, _ = \
    al_utils.permutation_importance_helper(
        estimator_names, 
        grid_search_cv_results_df, 
        train_cap_x_df, 
        train_y_df, 
        scoring = ['average_precision', 'roc_auc'],
        stop_reporting_threshold=0
)

## tune classification threshold for classification with classification metrics

### select the threshold

### check out the classifier as a function of classification threshold - select the classfication threshold at which the classifier will operated to make predictions

In [ ]:
validation_df = pd.read_csv('validation_df.csv').set_index(keys='index')
validation_df.index.name = None
validation_cap_x_df, validation_y_df = validation_df.iloc[:, :-1], validation_df.iloc[:, -1].to_frame()

In [ ]:
class_threshold_list = np.arange(0, 1.1, 0.1)
thresh_class_perf_dict = \
    class_utils.class_thresh_metrics_class_perf_assess_binary(
        best_model, 
        estimator_names, 
        grid_search_cv_results_df, 
        validation_cap_x_df, 
        validation_y_df, 
        class_threshold_list, 
        cvs_compute=False, 
        cvs_print=False, 
        data_set_name='validation', 
        model_selection_stage='tuned'
    )

In [ ]:
class_threshold_list = np.arange(0, 1.01, 0.01)
class_utils.plot_errors_as_a_function_of_classification_threshold(
    best_model, 
    estimator_names, 
    grid_search_cv_results_df,
    validation_cap_x_df, 
    validation_y_df, 
    class_threshold_list, 
    data_set_name='validation',
    model_selection_stage='tuned'
)

### set the classification threshold

In [ ]:
classification_threshold = 0.09

### use bootstrapping to understand how the precision will vary at this classification threshold

In [ ]:
class_utils.precision_recall_bootstrap_no_refit_binary(
    estimator_names, 
    grid_search_cv_results_df,
    validation_cap_x_df,
    validation_y_df, 
    n_bootstrap=20,
    data_set_name='validation', 
    model_selection_stage='tuned',
    classification_threshold=classification_threshold
)

In [ ]:
class_utils.roc_curve_bootstrap_no_refit_binary(
    estimator_names, 
    grid_search_cv_results_df, 
    validation_cap_x_df, 
    validation_y_df, 
    n_bootstrap=20,
    data_set_name='validation', 
    model_selection_stage='tuned',
    classification_threshold=classification_threshold
)

### can we do better? higher resolution scan of classification threshold around selected classification threshold

In [ ]:
start = classification_threshold - 0.05
stop = classification_threshold + 0.06
step_size = 0.01

class_threshold_list = np.arange(start, stop, step_size)
thresh_class_perf_dict = \
    class_utils.class_thresh_metrics_class_perf_assess_binary(
        best_model, 
        estimator_names, 
        grid_search_cv_results_df, 
        validation_cap_x_df, 
        validation_y_df, 
        class_threshold_list, 
        cvs_compute=False, 
        cvs_print=False, 
        data_set_name='validation', 
        model_selection_stage='tuned'
    )

### examine some classifier evaluations to better understand how the classiifcation metrics vary with classification threshold

In [ ]:
best_estimator = \
    grid_search_cv_results_df.loc[grid_search_cv_results_df.estimator == best_model, 'best_estimator'].iloc[0]

In [ ]:
class_perf_dict = class_utils.classification_performance(
    best_estimator, 
    validation_cap_x_df, 
    validation_y_df.values.ravel(), 
    classification_threshold=classification_threshold,
    binary=True,
    # https://scikit-learn.org/stable/modules/model_evaluation.html
    cvs_scoring_dict={
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall',
        'f1': 'f1'
    },
    cr_digits=4,
    cr_print=True,  # print classification report
    cm_print=True,  # print confusion matrix
    cvs_compute=False,  # compute cross_val_scores (classification threshold = 0.5 always)
    cvs_print=True,  # print cross_val_scores (classification threshold = 0.5 always) - ignored if cvs_compute=False
    prc_print=True,  # print precision and recall curves as a function of classification threshold
    prd_print=True,  # print precision recall curves
    roc_print=True,  # print roc curve
    data_set_name='validation', 
    model_selection_stage='tuned'
)

In [ ]:
del validation_cap_x_df, validation_y_df

## serialize model and classification threshold

In [ ]:
now = datetime.datetime.now()
date_time_prefix = str(now).replace('-', '_').replace(' ', '_').replace(':', '_').replace('.', '_')[:-4]

date_time_prefix

In [ ]:
best_estimator_file_name = date_time_prefix + '_cancer_screening_model' + '.pkl'

best_estimator_file_name

In [ ]:
best_estimator = \
    grid_search_cv_results_df.loc[grid_search_cv_results_df.estimator == best_model, 'best_estimator'].iloc[0]

model_dict = {
    'classification_threshold': classification_threshold,
    'best_model': best_model,
    'composite_estimator': best_estimator
}

In [ ]:
with open(best_estimator_file_name, 'wb') as f:
    pickle.dump(model_dict, f)

## evaluate model on the test set

This should be done in an independent notebook.